In [1]:
import pandas as pd
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.svm import LinearSVC
from sklearn.preprocessing import OrdinalEncoder
import random


In [7]:
df_font_names = pd.read_csv("../../Datasets/CharacterFontImages/font.names", header=None)
frames = []
for font_name in df_font_names[0]:
    df = pd.read_csv(f"../../Datasets/CharacterFontImages/{font_name}")
    frames.append(df)
df = pd.concat(frames)


In [21]:
ordinal_encoder = OrdinalEncoder()

for column in df.columns:
    if df[column].dtype == "object":
        df[column] = ordinal_encoder.fit_transform(df[column].values.reshape(-1, 1))

df.head()


,font,fontVariant,m_label,strength,italic,orientation,m_top,m_left,originalH,originalW,...,r19c10,r19c11,r19c12,r19c13,r19c14,r19c15,r19c16,r19c17,r19c18,r19c19
0,0.0,0.0,64258,0.4,0,0.0,35,21,51,22,...,1,1,1,1,1,1,163,255,255,255
1,0.0,0.0,64257,0.4,0,0.0,35,21,51,22,...,1,1,1,1,1,1,163,255,255,255
2,0.0,0.0,61442,0.4,0,0.0,35,21,51,22,...,1,1,1,1,1,1,163,255,255,255
3,0.0,0.0,61441,0.4,0,0.0,35,21,51,22,...,1,1,1,1,1,1,163,255,255,255
4,0.0,0.0,9674,0.4,0,0.0,51,21,33,25,...,255,132,1,1,1,1,1,1,1,1


In [33]:
X = df.iloc[random.sample(range(df.shape[0]), int(df.shape[0] * 0.0001)),
            random.sample(range(1, df.shape[1]), int(df.shape[1] * 0.1))]
y = df.iloc[random.sample(range(df.shape[0]), int(df.shape[0] * 0.0001)), :1]


In [34]:
import warnings
warnings.filterwarnings("ignore")

linear_svc = LinearSVC()
forward_selector = SequentialFeatureSelector(
    linear_svc, scoring="accuracy", n_features_to_select=39, direction="backward")
forward_selector.fit(X, y.values.ravel())

selected_features_mask = forward_selector.get_support()
selected_feature_indices = [i for (i, x) in enumerate(selected_features_mask) if x]

print("Before Backward feature elimination:", df.shape)
print("After Backward feature elimination:", df.iloc[:, selected_feature_indices + [-1]].shape)

df.iloc[:, [0] + selected_feature_indices].head()


Before Backward feature elimination: (832670, 412)
After Backward feature elimination: (832670, 40)


,font,font,fontVariant,m_label,strength,italic,m_top,m_left,originalH,originalW,...,r0c19,r1c0,r1c1,r1c2,r1c3,r1c4,r1c5,r1c6,r1c7,r1c8
0,0.0,0.0,0.0,64258,0.4,0,35,21,51,22,...,255,1,1,1,93,255,255,255,176,146
1,0.0,0.0,0.0,64257,0.4,0,35,21,51,22,...,65,1,1,1,93,255,255,255,176,146
2,0.0,0.0,0.0,61442,0.4,0,35,21,51,22,...,255,1,1,1,93,255,255,255,176,146
3,0.0,0.0,0.0,61441,0.4,0,35,21,51,22,...,65,1,1,1,93,255,255,255,176,146
4,0.0,0.0,0.0,9674,0.4,0,51,21,33,25,...,1,1,1,1,1,1,1,1,38,255
